# 第14章：もうちょっとだけモナド

Maybeモナド・リストもなど・IOモナドを今までやってきた  
さらにいくつかのモナドがこの章で出てくる  

この章で出てくるのはmtlパッケージ  
インストールされているかは `ghc-pkg list` とうってmtlとバージョン番号が出るはず  

### Writerの話

In [1]:
isBigGang :: Int -> Bool
isBigGang x = x > 9

In [2]:
isBigGang :: Int -> (Bool, String)
isBigGang x = (x > 9, "Compared gang size to 9.")

In [3]:
isBigGang 3

(False,"Compared gang size to 9.")

例えば既に文脈がついた値 `(3, "Smallish gang.")` をisBigGangに食わせたかったらどうするか  
ログのついた値、`(a, String)` 型の値と `a -> (b, String)` 型の関数2つを取り、その値を関数の方に食わせる関数を作る  

In [4]:
applyLog :: (a, String) -> (a -> (b, String)) -> (b, String)
applyLog (x, log) f = let (y, newLog) = f x in (y, log ++ newLog)

In [6]:
(3, "Smallish gang.") `applyLog` isBigGang

(False,"Smallish gang.Compared gang size to 9.")

In [7]:
("Tobin", "Got outlaw name.") `applyLog` (\x -> (length x, "applied length"))

(5,"Got outlaw name.applied length")

### モノイドが助けに来たよ

Logは別にStringである必要はない  


In [1]:
applyLog :: (Monoid m) => (a, m) -> (a -> (b, m)) -> (b, m)
applyLog (x, log) f = let (y, newLog) = f x in (y, log `mappend` newLog)


In [4]:
import Data.Monoid

type Food = String
type Price = Sum Int

addDrink :: Food -> (Food, Price)
addDrink "beans" = ("milk", Sum 25)
addDrink "jerky" = ("whiskey", Sum 99)
addDrink _ = ("beer", Sum 30)


In [3]:
Sum 3 `mappend` Sum 9

Not in scope: data constructor `Sum'
Perhaps you meant variable `sum' (imported from Prelude)


Not in scope: data constructor `Sum'
Perhaps you meant variable `sum' (imported from Prelude)

In [2]:
("beans", Sum 10) `applyLog` addDrink

Not in scope: data constructor `Sum'
Perhaps you meant variable `sum' (imported from Prelude)


Not in scope: `addDrink'

### Writer型

```
newtype Writer w a = Writer {runWriter :: (a, w)}
```

In [6]:
import Control.Monad.Writer

In [8]:
runWriter (return 3 :: Writer String Int)

(3,"")

In [9]:
runWriter (return 3 :: Writer String Int)

(3,"")

In [10]:
    runWriter (return 3 :: Writer (Product Int) Int)

(3,Product {getProduct = 1})

In [11]:
logNumber :: Int -> Writer [String] Int 
logNumber x = writer (x, ["Got number: " ++ show x]) 

multWithLog :: Writer [String] Int
multWithLog = do 
    a <- logNumber 3 
    b <- logNumber 5 
    return (a* b)

In [12]:
logNumber 1

WriterT (Identity (1,["Got number: 1"]))

In [14]:
runWriter multWithLog

(15,["Got number: 3","Got number: 5"])

In [15]:
multWithLog :: Writer [String] Int
multWithLog = do
    a <- logNumber 3 
    b <- logNumber 5 
    tell ["Gonna multiply these two"] 
    return (a* b)

In [16]:
runWriter multWithLog

(15,["Got number: 3","Got number: 5","Gonna multiply these two"])

In [17]:
gcd' :: Int -> Int -> Writer [String] Int

gcd' a b
    | b == 0 = do
        tell ["Finished with " ++ show a] 
        return a
    | otherwise = do
        tell [show a ++ " mod " ++ show b ++ " = " ++ show (a `mod ` b)] 
        gcd' b (a ` mod ` b)

In [18]:
gcd' 8 3

WriterT (Identity (1,["8 mod 3 = 2","3 mod 2 = 1","2 mod 1 = 0","Finished with 1"]))

In [19]:
fst $ runWriter (gcd' 8 3)

1

In [20]:
mapM_ putStrLn $ snd $ runWriter (gcd' 8 3)

8 mod 3 = 2
3 mod 2 = 1
2 mod 1 = 0
Finished with 1

### 非効率なリスト構築

In [22]:
gcdReverse :: Int -> Int -> Writer [String] Int 
gcdReverse a b
    | b == 0 = do 
        tell ["Finished with " ++ show a] 
        return a 
    | otherwise = do 
        result <- gcdReverse b (a ` mod ` b) 
        tell [show a ++ " mod " ++ show b   ++ " = " ++  show (a ` mod ` b)] 
        return result
        

In [23]:
mapM_ putStrLn $ snd $ runWriter (gcdReverse 8 3)

Finished with 1
2 mod 1 = 0
3 mod 2 = 1
8 mod 3 = 2

In [1]:
newtype DiffList a = DiffList {getDiffList :: [a] -> [a]}

In [2]:
toDiffList :: [a] -> DiffList a
toDiffList xs = DiffList (xs++)

fromDiffList :: DiffList a -> [a]
fromDiffList (DiffList f) = f []

In [3]:
instance Monoid (DiffList a) where
    mempty = DiffList (\xs -> [] ++ xs)
    (DiffList f) `mappend` (DiffList g) = DiffList (\xs -> f (g xs))

Line 3: Avoid lambda
Found:
\ xs -> f (g xs)
Why not:
f . g

In [27]:
fromDiffList (toDiffList [1, 2, 3, 4] ` mappend ` toDiffList [1, 2, 3])

[1,2,3,4,1,2,3]

In [30]:
gcd' :: Int -> Int -> Writer (DiffList String) Int
gcd' a b
    | b == 0 = do 
        tell (toDiffList ["Finished with " ++ show a]) 
        return a 
    | otherwise = do 
        result <- gcd' b (a ` mod ` b)
        tell (toDiffList [show a ++ " mod " ++ show b   ++ " = " ++  show (a ` mod ` b)] )
        return result

In [32]:
mapM_ putStrLn . fromDiffList . snd . runWriter $ gcd' 110 34

Finished with 2
8 mod 2 = 0
34 mod 8 = 2
110 mod 34 = 8

In [2]:
import Control.Monad.Writer

newtype DiffList a = DiffList {getDiffList :: [a] -> [a]}

toDiffList :: [a] -> DiffList a
toDiffList xs = DiffList (xs++)

fromDiffList :: DiffList a -> [a]
fromDiffList (DiffList f) = f []

instance Monoid (DiffList a) where
    mempty = DiffList (\xs -> [] ++ xs)
    (DiffList f) `mappend` (DiffList g) = DiffList (\xs -> f (g xs))

finalCountDown :: Int -> Writer (DiffList String) ()
finalCountDown 0 = do
    tell (toDiffList ["0"])
finalCountDown x = do
    finalCountDown (x-1)
    tell (toDiffList [show x])


In [3]:
mapM_ putStrLn . fromDiffList . snd . runWriter $ finalCountDown 5

0
1
2
3
4
5

In [8]:
import Control.Monad.Writer

newtype DiffList a = DiffList {getDiffList :: [a] -> [a]}

toDiffList :: [a] -> DiffList a
toDiffList xs = DiffList (xs++)

fromDiffList :: DiffList a -> [a]
fromDiffList (DiffList f) = f []

instance Monoid (DiffList a) where
    mempty = DiffList (\xs -> [] ++ xs)
    (DiffList f) `mappend` (DiffList g) = DiffList (\xs -> f (g xs))

finalCountDown' :: Int -> Writer [String] ()
finalCountDown' 0 = do
    tell ["0"]
finalCountDown' x = do
    finalCountDown (x-1)
    tell  [show x]


In [ ]:
mapM_ putStrLn . snd . runWriter $ finalCountDown' 50000

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35

### Reader?それはあなたです!



In [1]:
f = (*5)
g = (+3)
(fmap f g) 8

Line 1: Redundant bracket
Found:
(fmap f g) 8
Why not:
fmap f g 8

55

In [2]:
f = (+) <$> (*2) <*> (+10)

In [3]:
f 3

19

In [1]:
type Stack = [Int]

pop :: Stack -> (Int, Stack)
pop (x:xs) = (x, xs)

push :: Int -> Stack -> ((), Stack)
push a xs = ((), a : xs)


In [2]:
stackManip :: Stack -> (Int, Stack)
stackManip stack = let
    ((), newStack1) = push 3 stack
    (a, newStack2) = pop newStack1
    in pop newStack2

In [3]:
stackManip [5,8,2,1]

(5,[8,2,1])

```
stackManip = do
    push 3
    a <- pop
    pop
```

とかけたらかっこいいよねそれが、Stateモナドならかける

In [4]:
import Control.Monad.State

pop :: State Stack Int
pop = state $ \(x:xs) -> (x, xs)

push :: Int -> State Stack ()
push a = state $ \xs -> ((), a:xs)


In [5]:
import Control.Monad.State

stackManip :: State Stack Int
stackManip = do
    push 3
    a <- pop
    pop

In [6]:
runState stackManip [5,8,2,1]

(5,[8,2,1])

In [7]:
:t Right 4

Right 4 :: forall a b. Num b => Either a b

In [8]:
:t Either a b

Not in scope: data constructor `Either'
Perhaps you meant variable `either' (imported from Prelude)


Not in scope: `a'


Not in scope: `b'

In [9]:
:t Either

Not in scope: data constructor `Either'
Perhaps you meant variable `either' (imported from Prelude)

In [10]:
:t Left "aaa"

Left "aaa" :: forall b. Either [Char] b

In [11]:
:t strMsg

Not in scope: `strMsg'

In [12]:
Left "boom" >>= \x -> return (x+1)

Left "boom"

In [13]:
Right 100 >>= \x -> return (x+1)

Right 101

In [14]:
Right 3 >>= \x -> return (x + 100) :: Either String Int

Right 103

In [15]:
liftM (*3) (Just 7)

Just 21

In [16]:
:t lift

lift :: forall (t :: (* -> *) -> * -> *) (m :: * -> *) a. (Monad m, MonadTrans t) => m a -> t m a

In [17]:
:t liftM

liftM :: forall a1 r (m :: * -> *). Monad m => (a1 -> r) -> m a1 -> m r

In [18]:
lift (*5) 5

Couldn't match kind `* -> *' with `*'
When matching types
  t0 :: (* -> *) -> * -> *
  (->) :: * -> * -> *
Expected type: a0 -> a
  Actual type: t0 ((->) a) a
The function `lift' is applied to two arguments,
but its type `(a -> a) -> t0 ((->) a) a' has only one
In the expression: lift (* 5) 5
In an equation for `it': it = lift (* 5) 5

In [20]:
:t ap

ap :: forall (m :: * -> *) a b. Monad m => m (a -> b) -> m a -> m b

In [21]:
join (Just (Just 9))

Just 9

In [22]:
join (Just Nothing)

Nothing

In [24]:
filter (\x -> x `mod` 2 == 0) [1,2,3,4,5]

[2,4]

In [26]:
filterM (\x -> Just(x `mod` 2 == 0)) [1,2,3,4,5]

Just [2,4]

In [27]:
foldl (\acc x -> acc + x) 0 [2,8,3,1]

Line 1: Avoid lambda
Found:
\ acc x -> acc + x
Why not:
(+)

14

In [28]:
binSmalls :: Int -> Int -> Maybe Int
binSmalls acc x
    | x > 9 = Nothing
    | otherwise = Just (acc + x)

In [29]:
foldM binSmalls 0 [2,8,3,1]

Just 14

### モナドを作る

In [37]:
a = [(3, 0.5), (5,0.25), (9, 0.25)]


In [32]:
import Data.Ratio
1 % 4

1 % 4

In [33]:
1 % 2 + 1 % 2

1 % 1

In [38]:
a = [(3, 1%2), (5,1%4), (9, 1%4)]

In [36]:
import Data.Ratio

newtype Prob a = Prob { getProb :: [(a, Rational)]} deriving Show

instance Functor Prob where
    fmap f (Prob xs) = Prob $ map (\(x, p) -> (f x, p)) xs

In [39]:
fmap negate (Prob a)

Prob {getProb = [(-3,1 % 2),(-5,1 % 4),(-9,1 % 4)]}

In [41]:
thisSituation :: Prob (Prob Char)
thisSituation = Prob
    [(Prob [('a', 1%2), ('b', 1 %2)], 1 % 4)
    , (Prob [('b', 1 % 2), ('d', 1 % 2)], 3 % 4)
    ]

In [42]:
flatten :: Prob (Prob a) -> Prob a
flatten (Prob xs) = Prob $ concat $ map multAll xs
    where multAll (Prob innerxs, p) = map (\(x, r) -> (x, p * r)) innerxs

Line 2: Use concatMap
Found:
concat $ map multAll xs
Why not:
concatMap multAll xsLine 3: Use second
Found:
\ (x, r) -> (x, p * r)
Why not:
Control.Arrow.second ((*) p)

In [43]:
instance Monad Prob where
    return x = Prob [(x, 1 % 1)]
    m >>= f = flatten (fmap f m)
    fail _ = Prob []

No instance for (Applicative Prob) arising from the superclasses of an instance declaration
In the instance declaration for `Monad Prob'